<a href="https://colab.research.google.com/github/TamannaAhmad/research-paper-analyser/blob/main/image_captioning_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=caf5e5de4513183d1d80814872f28200d0e45d695fae6b422ecc86031d8b987c
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from transformers import BlipProcessor, BlipForConditionalGeneration
from evaluate import load
from PIL import Image
from tqdm import tqdm
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class PaperImageCaptionDataset(Dataset):
    def __init__(self, data_dir, transform=None, max_samples=None):
        self.data_dir = data_dir
        self.transform = transform
        self.samples = []

        # Get subdirectories for each paper
        papers = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

        # Collect valid image-caption pairs without loading images
        for paper in papers:
            paper_dir = os.path.join(data_dir, paper)
            files = os.listdir(paper_dir)

            image_files = [f for f in files if os.path.splitext(f)[1].lower() in ['.png', '.jpg', '.jpeg']]

            for img_file in image_files:
                base_name = os.path.splitext(img_file)[0]
                caption_file = f"{base_name}_caption.txt"

                if caption_file in files:
                    img_path = os.path.join(paper_dir, img_file)
                    caption_path = os.path.join(paper_dir, caption_file)

                    try:
                        with open(caption_path, 'r', encoding='utf-8') as f:
                            caption = f.read().strip()
                            if caption:
                                self.samples.append((img_path, caption))
                    except Exception as e:
                        print(f"Error reading caption {caption_path}: {e}")

        # Limit dataset size if specified
        if max_samples and len(self.samples) > max_samples:
            self.samples = self.samples[:max_samples]

        print(f"Dataset created with {len(self.samples)} image-caption pairs")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, caption = self.samples[idx]

        try:
            # Only load image when requested during iteration
            image = Image.open(img_path).convert('RGB')

            if self.transform:
                image = self.transform(image)

            return {'image': image, 'caption': caption, 'path': img_path}

        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a placeholder
            if self.transform:
                placeholder = torch.zeros(3, 384, 384)
            else:
                placeholder = Image.new('RGB', (384, 384), color='black')

            return {'image': placeholder, 'caption': "Error loading image", 'path': img_path}

In [ ]:
# data preparation
def create_dataloaders(data_dir, batch_size=4, image_size=384, train_split=0.9):
    # image transformations
    transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

    # create dataset
    dataset = PaperImageCaptionDataset(data_dir, transform=transform)

    # check if dataset is empty
    if len(dataset) == 0:
        raise ValueError("Dataset is empty. Check that your image-caption pairs were properly extracted.")

    # split into train and validation
    train_size = int(train_split * len(dataset))
    val_size = len(dataset) - train_size

    if train_size == 0 or val_size == 0:
        raise ValueError(f"Not enough data for splitting. Found {len(dataset)} samples.")

    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    # create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    return train_loader, val_loader

In [ ]:
def evaluate_model(model, processor, dataloader, device="cuda"):
    model.eval()
    references = []  # List of lists (each sublist contains reference captions)
    hypotheses = []  # List of generated captions

    # Load metrics
    bleu = load('bleu')
    rouge = load('rouge')
    meteor = load('meteor')

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            images = batch['image'].to(device)
            original_captions = batch['caption']

            # Skip batches with errors
            if "Error loading image" in original_captions:
                continue

            try:
                # Generate captions
                generated_ids = model.generate(
                    pixel_values=images,
                    max_length=100,
                    num_beams=5,
                    no_repeat_ngram_size=2,
                    temperature=1.2,
                    do_sample=True
                )
                generated_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                hypotheses.extend(generated_captions)
                references.extend([[cap] for cap in original_captions])  # Wrap each caption in a list

            except Exception as e:
                print(f"Error generating captions: {e}")
                continue

    # Compute metrics
    bleu_results = bleu.compute(predictions=hypotheses, references=references)
    rouge_results = rouge.compute(predictions=hypotheses, references=references)
    meteor_results = meteor.compute(predictions=hypotheses, references=references)

    print("\nEvaluation Results:")
    print(f"BLEU: {bleu_results['bleu']:.4f}")
    print(f"ROUGE-L: {rouge_results['rougeL']:.4f}")
    print(f"METEOR: {meteor_results['meteor']:.4f}")

    return {
        'bleu': bleu_results,
        'rouge': rouge_results,
        'meteor': meteor_results
    }

In [ ]:
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Paths (update with your paths)
    data_dir = "/content/drive/MyDrive/training_data_images"
    model_path = "/content/drive/MyDrive/best_blip_captioning_model.pth"

    # Create dataloaders
    _, val_loader = create_dataloaders(data_dir, batch_size=4)

    # Load model and processor
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)

    # Run evaluation
    results = evaluate_model(model, processor, val_loader, device)

In [ ]:
if __name__ == "__main__":
    main()

Using device: cuda
Dataset created with 440 image-caption pairs


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

<ipython-input-7-62baf9f1d6a9>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
Evaluating: 100%|██████████| 11/11 [02:43<00:00, 14.85s/it]



Evaluation Results:
BLEU: 0.0000
ROUGE-L: 0.1447
METEOR: 0.1037
